In [1]:
import os
import pandas as pd
os.chdir(r'C:\Users\liedt\OneDrive\Georgia Tech\CSE 6242 Data and Visual Analytics\Project\model')


In [2]:
# Load data, rename cols, and set index
# A single zip maps to multiple tract codes. Goal is aggregate by zip, as this is the level in the primary dataset

# Source: https://www.huduser.gov/portal/datasets/usps_crosswalk.html
df_zip_tract = pd.read_excel('ZIP_TRACT_092019.xlsx', converters={'zip': lambda x: str(x)})

# Source: https://www.ers.usda.gov/data-products/rural-urban-commuting-area-codes.aspx
df_tract_details = pd.read_excel('ruca2010revised.xlsx', sheet_name = 'Data', skiprows = 1)

df_tract_details_cols = {
    'State-County FIPS Code': 'state_County_FIPS_Code',
    'Select State': 'state',
    'Select County': 'county',
    'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)': 'state_county_tract_FIPS_code',
    'Primary RUCA Code 2010': 'primary_RUCA_code_2010',
    'Secondary RUCA Code, 2010 (see errata)': 'secondary_RUCA_code_2010',
    'Tract Population, 2010': 'tract_population_2010',
    'Land Area (square miles), 2010': 'tract_land_area_2010',
    'Population Density (per square mile), 2010': 'tract_population_density_2010',
}

df_tract_details.rename(columns=df_tract_details_cols, inplace=True)

df_tract_details = df_tract_details.set_index('state_county_tract_FIPS_code')
df_zip_tract = df_zip_tract.set_index('tract')

In [3]:
# data aggregation

df_join = df_zip_tract.join(df_tract_details)
    
df_agg = df_join.groupby('zip', as_index=False).agg(
    {"tract_population_2010": "sum",
    "tract_land_area_2010": "sum",
    "primary_RUCA_code_2010": "max",
    }
)

df_agg = df_join.groupby('zip').agg(
    zip_population_2010 = ("tract_population_2010", "sum"),
    zip_land_area_2010 = ("tract_land_area_2010", "sum"),
    primary_RUCA_code_2010 = ("primary_RUCA_code_2010", "min")
)

df_agg['zip_population_density_2010'] = df_agg['zip_population_2010'] / df_agg['zip_land_area_2010']

# Source: Medicare Part D Opioid Prescribing Mapping Tool Methodology.pdf
df_agg['rural_urban_class'] = [
    'urban' if 1 <= x <= 3 else 
    'rural' if 4 <= x <= 10 else 
    'other' for x in df_agg['primary_RUCA_code_2010']
]

df_agg = df_agg.reset_index()

In [4]:
# Preview
df_agg.head()

,zip,zip_population_2010,zip_land_area_2010,primary_RUCA_code_2010,zip_population_density_2010,rural_urban_class
0,00501,3355.0,1.093031,1.0,3069.446526,urban
1,00601,15448.0,35.441027,4.0,435.879015,rural
2,00602,15098.0,10.175260,1.0,1483.795025,urban
3,00603,44653.0,21.209546,1.0,2105.325617,urban
4,00604,3687.0,3.989034,1.0,924.284031,urban


In [5]:
df_agg.to_csv('zip_reference.csv', index = False)